<a href="https://colab.research.google.com/github/1ucky40nc3/easy_mpt/blob/main/examples/notebooks/run_mpt_with_ctransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install Requirements
!pip install einops
!pip install transformers
!pip install accelerate
!pip install sentencepiece
!pip install ctransformers

!git lfs install

In [ ]:
# @title Clone the ggml Repository
!git clone https://github.com/ggerganov/ggml

In [ ]:
# @title Build the ggml MPT Features
!mkdir -p /content/ggml/build
%cd /content/ggml/build
!cmake ..
!make -j4 mpt mpt-quantize

In [ ]:
# @title Download the `mosaicml/mpt-7b-instruct` PLM
!mkdir -p /content/models
%cd /content/models
!git clone https://huggingface.co/mosaicml/mpt-7b-instruct

In [ ]:
# @title Convert the PLM into the ggml Format
%cd /content/ggml/build
!python ../examples/mpt/convert-h5-to-ggml.py /content/models/mpt-7b-instruct 0

In [ ]:
# @title Quantize the PLM
%cd /content/ggml/build
!./bin/mpt-quantize \
    /content/models/mpt-7b-instruct/ggml-model-f32.bin \
    /content/models/mpt-7b-instruct/model-quant.bin \
    q4_0

In [ ]:
# @title Import Dependencies
from ctransformers import AutoModelForCausalLM

In [ ]:
# @title Load the GGML **MosaicML MPT-7B-Instruct** model in *fp32*
llm_fp32 = AutoModelForCausalLM.from_pretrained(
    "/content/models/mpt-7b-instruct/ggml-model-f32.bin", 
    model_type="mpt"
)

In [ ]:
# @title Do some Inference
llm_fp32("AI is going to")

In [ ]:
# @title Load the *Quantized* GGML **MosaicML MPT-7B-Instruct**
llm_quant = AutoModelForCausalLM.from_pretrained(
    "/content/models/mpt-7b-instruct/model-quant.bin", 
    model_type="mpt"
)

In [ ]:
# @title Do some Inference
llm_quant("AI is going to")